## Hacking 4 - Deepseek MLA Walkthrough simplified
- Ignore setup and naive method right now, got back to these as needed
- Here just focus on understanding what's happening with the MLA forward pass.
- Oh wow looks like it might kinda be possible to just load a subset of data from the safetensors?!
- Hmm that's pretty interesting - distraction or a good use of time? Let me poke at it for a couplte minutes here.
- If i wanted all the layer6 atention stuff, it looks like I would just need

```
"model.layers.6.self_attn.q_a_proj.weight": "model-00009-of-000163.safetensors"
```

And layer 0 attention stuff would just be: 

```
"model.layers.0.self_attn.q_a_proj.weight": "model-00001-of-000163.safetensors"
```
- Ok I have a theory about why I can't load weights -> I bet it's because the hf and github repos -> eh that doesn't seem to be it. 
- Tried manually mapping too -> ran into size issues.
- Ok let me push a little further here and try a "clean start" from the hf repo"
- Ok yeah I think the step i've been missing is "convert.py"

In [1]:
import matplotlib.pyplot as plt

In [2]:
from transformers import AutoTokenizer
import torch

import sys
# sys.path.append('DeepSeek-V3/inference') #Github slightly newer
sys.path.append('DeepSeek-V3/DeepSeek-V3/inference') #Hugging face

from model import Transformer, MLA, ModelArgs, apply_rotary_emb

In [3]:
args=ModelArgs(**{"vocab_size": 129280,
                "dim": 7168,
                "inter_dim": 18432,
                "moe_inter_dim": 2048,
                "n_layers": 61,
                "n_dense_layers": 3,
                "n_heads": 128,
                "n_routed_experts": 256,
                "n_shared_experts": 1,
                "n_activated_experts": 8,
                "n_expert_groups": 8,
                "n_limited_groups": 4,
                "route_scale": 2.5,
                "score_func": "sigmoid",
                "q_lora_rank": 1536,
                "kv_lora_rank": 512,
                "qk_nope_head_dim": 128,
                "qk_rope_head_dim": 64,
                "v_head_dim": 128,
                "dtype": "bf16"}) #fp8 seems out due to my hardware? bf16

In [4]:
model=Transformer(args)

### Let me spend a few minutes seeing if I can load just the attention weights for the embedding, layers 0 and maybe layer 6

In [5]:
from safetensors.torch import load_model, load_file
import os

In [7]:
# ckpt_path='/home/stephen/deepseek/DeepSeek-V3/DeepSeek-V3'
ckpt_path='/home/stephen/deepseek/clean_hf_pull/DeepSeek-V3/weights_converted'
# rank=0
# world_size=1

#
# missing, unexpected=load_model(model, os.path.join(ckpt_path, 'model-00001-of-000163.safetensors'), strict=False)
# load_model(model, os.path.join(ckpt_path, 'model-00009-of-000163.safetensors'), strict=False)
missing, unexpected=load_model(model, os.path.join(ckpt_path, "model0-mp1.safetensors"), strict=False) #Result of conversion process

In [8]:
len(missing)

44960

In [9]:
len(unexpected)

411

In [10]:
state_dict = load_file(os.path.join(ckpt_path, "model0-mp1.safetensors"), device='cpu')

In [11]:
state_dict.keys()

dict_keys(['embed.weight', 'layers.0.attn.kv_norm.weight', 'layers.0.attn.q_norm.weight', 'layers.0.attn.wkv_a.scale', 'layers.0.attn.wkv_a.weight', 'layers.0.attn.wkv_b.scale', 'layers.0.attn.wkv_b.weight', 'layers.0.attn.wo.scale', 'layers.0.attn.wo.weight', 'layers.0.attn.wq_a.scale', 'layers.0.attn.wq_a.weight', 'layers.0.attn.wq_b.scale', 'layers.0.attn.wq_b.weight', 'layers.0.attn_norm.weight', 'layers.0.ffn.w1.scale', 'layers.0.ffn.w1.weight', 'layers.0.ffn.w2.scale', 'layers.0.ffn.w2.weight', 'layers.0.ffn.w3.scale', 'layers.0.ffn.w3.weight', 'layers.0.ffn_norm.weight', 'layers.1.attn.kv_norm.weight', 'layers.1.attn.q_norm.weight', 'layers.1.attn.wkv_a.scale', 'layers.1.attn.wkv_a.weight', 'layers.1.attn.wkv_b.scale', 'layers.1.attn.wkv_b.weight', 'layers.1.attn.wo.scale', 'layers.1.attn.wo.weight', 'layers.1.attn.wq_a.scale', 'layers.1.attn.wq_a.weight', 'layers.1.attn.wq_b.scale', 'layers.1.attn.wq_b.weight', 'layers.1.attn_norm.weight', 'layers.1.ffn.w1.scale', 'layers.1.ffn

In [12]:
model_state_dict=model.state_dict()

In [14]:
# model_state_dict.keys()

In [16]:
len(state_dict)

846

In [15]:
missing, unexpected = model.load_state_dict(state_dict, strict=False)

In [18]:
len(unexpected) #Ok some are getting loaded -> maybe the ones I need are?

411

In [19]:
model.layers[0].attn.wq_a.weight.detach().abs().sum()

tensor(7.5078e+08, dtype=torch.bfloat16)

Numbers!!!

In [21]:
model.layers[0].attn.wq_a.weight

Parameter containing:
tensor([[ -22.0000,  -72.0000,   88.0000,  ...,   -9.0000, -208.0000,
          -28.0000],
        [ 128.0000,   14.0000,   16.0000,  ...,  104.0000,  -64.0000,
           26.0000],
        [  72.0000,  -36.0000,   64.0000,  ..., -120.0000,   80.0000,
          -72.0000],
        ...,
        [-144.0000,   80.0000,   48.0000,  ...,  -72.0000,  -96.0000,
           72.0000],
        [ -80.0000,  120.0000,   72.0000,  ...,  -44.0000,  112.0000,
          112.0000],
        [ 224.0000,    4.5000,  -56.0000,  ...,  160.0000,  -64.0000,
           36.0000]], dtype=torch.bfloat16, requires_grad=True)

In [25]:
# model.layers[0].attn.wq_a.bias

In [27]:
model.embed.weight

Parameter containing:
tensor([[-2.3804e-02,  2.5879e-02,  1.7334e-02,  ..., -3.6377e-02,
         -1.7334e-02, -1.0437e-02],
        [-2.1240e-02,  2.8076e-02,  1.0400e-01,  ..., -1.3504e-03,
         -2.0874e-02, -1.1230e-02],
        [-1.4465e-02,  1.0315e-02,  1.4771e-02,  ...,  2.8076e-03,
          3.1738e-02, -1.9897e-02],
        ...,
        [ 1.4725e-03, -8.0566e-03,  1.5488e-03,  ..., -1.9379e-03,
         -4.6539e-04,  3.3112e-03],
        [ 4.7607e-03,  3.2196e-03, -5.9891e-04,  ..., -7.2098e-04,
         -2.3193e-03, -1.0986e-02],
        [ 5.3346e-06, -7.1716e-03,  6.7139e-04,  ..., -3.8605e-03,
         -6.1340e-03, -2.5635e-03]], requires_grad=True)

- OK NICE!
- Took some hacking, but I should be able to trace like actually values through this thing now
- Next I'll convert a few more tensors (into a new directory), make a clean from scratch notebook, and get to tracing and visualizing! Let's go!

`load_model` not quite working as expected, getting a little deeper. 
```
    state_dict = load_file(filename, device=device)
    model_state_dict = model.state_dict()
    to_removes = _remove_duplicate_names(model_state_dict, preferred_names=state_dict.keys())
    missing, unexpected = model.load_state_dict(state_dict, strict=False)
```

In [7]:
state_dict = load_file(os.path.join(ckpt_path, 'model-00001-of-000163.safetensors'), device='cpu')

In [8]:
model_state_dict=model.state_dict()

In [ ]:
#Statedict from safe tensor
'model.embed_tokens.weight', 
'model.layers.0.input_layernorm.weight', 
'model.layers.0.mlp.down_proj.weight', 
'model.layers.0.mlp.down_proj.weight_scale_inv', 
'model.layers.0.mlp.gate_proj.weight', 
'model.layers.0.mlp.gate_proj.weight_scale_inv', 
'model.layers.0.mlp.up_proj.weight',
'model.layers.0.mlp.up_proj.weight_scale_inv', 
'model.layers.0.post_attention_layernorm.weight', 

'model.layers.0.self_attn.kv_a_layernorm.weight', 
'model.layers.0.self_attn.kv_a_proj_with_mqa.weight', 
'model.layers.0.self_attn.kv_a_proj_with_mqa.weight_scale_inv', 
'model.layers.0.self_attn.kv_b_proj.weight', 
'model.layers.0.self_attn.kv_b_proj.weight_scale_inv', 

'model.layers.0.self_attn.o_proj.weight', 
'model.layers.0.self_attn.o_proj.weight_scale_inv',

'model.layers.0.self_attn.q_a_layernorm.weight', 
'model.layers.0.self_attn.q_a_proj.weight', 
'model.layers.0.self_attn.q_a_proj.weight_scale_inv', 
'model.layers.0.self_attn.q_b_proj.weight', 
'model.layers.0.self_attn.q_b_proj.weight_scale_inv',  #Hmm not sure where to put these

'model.layers.1.input_layernorm.weight'

In [ ]:
#Statedict from model -> these names are super different dafuq? Man I hope I'm not missing something super essential
'embed.weight', 
'layers.0.attn.wq_a.weight', 
'layers.0.attn.q_norm.weight', 
'layers.0.attn.wq_b.weight', 
'layers.0.attn.wkv_a.weight', 
'layers.0.attn.kv_norm.weight', 
'layers.0.attn.wkv_b.weight', 
'layers.0.attn.wo.weight', 
'layers.0.ffn.w1.weight', 
'layers.0.ffn.w2.weight', 
'layers.0.ffn.w3.weight', 
'layers.0.attn_norm.weight', 
'layers.0.ffn_norm.weight'

In [22]:
# Ok this seems sketchy -> I'm kinda having to guess which thing is which -> maybe i try from scratch using
# hf repo? Does seem to be sortof working, let me push a little further and fall back to that if needed
my_state_dict={}
my_state_dict['embed.weight']=state_dict['model.embed_tokens.weight']
my_state_dict['layers.0.attn.wq_a.weight']=state_dict['model.layers.0.self_attn.q_a_proj.weight']
my_state_dict['layers.0.attn.q_norm.weight']=state_dict['model.layers.0.self_attn.q_a_layernorm.weight']
my_state_dict['layers.0.attn.wq_b.weight']=state_dict['model.layers.0.self_attn.q_b_proj.weight']

my_state_dict['layers.0.attn.wkv_a.weight'] =state_dict['model.layers.0.self_attn.kv_a_layernorm.weight']
my_state_dict['layers.0.attn.kv_norm.weight'] =state_dict['model.layers.0.self_attn.kv_a_layernorm.weight']
my_state_dict['layers.0.attn.wkv_b.weight'] =state_dict['model.layers.0.self_attn.kv_b_proj.weight']
my_state_dict['layers.0.attn.wo.weight'] =state_dict['model.layers.0.self_attn.o_proj.weight']

# my_state_dict['layers.0.ffn.w1.weight'] =state_dict[]
# my_state_dict['layers.0.ffn.w2.weight'] =state_dict[]
# my_state_dict['layers.0.ffn.w3.weight'] =state_dict[]
# my_state_dict['layers.0.attn_norm.weight'] =state_dict[]
# my_state_dict['layers.0.ffn_norm.weight']=state_dict[]

In [14]:
# state_dict.keys()

In [16]:
# model_state_dict.keys()

In [8]:
# state_dict.keys()

In [9]:
len(state_dict)

270

In [17]:
# state_dict['model.embed_tokens.weight']

In [23]:
# missing, unexpected = model.load_state_dict(state_dict, strict=False) #, assign=True)
missing, unexpected = model.load_state_dict(my_state_dict, strict=False) #, assign=True)

RuntimeError: Error(s) in loading state_dict for Transformer:
	size mismatch for layers.0.attn.wkv_a.weight: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([576, 7168]).

In [24]:
len(missing)

45392

In [20]:
len(unexpected)

0

In [21]:
model.embed

ParallelEmbedding()

In [28]:
# model.load_state_dict?

In [8]:
len(missing)

45395

In [9]:
len(unexpected)

270

In [15]:
model.layers[0].attn.wq_a.weight.detach().abs().sum()

tensor(0., dtype=torch.bfloat16)

In [42]:
# model.layers[6].attn.kv_a_layernorm.weight
# plt.imshow(model.embed.weight.detach().cpu().numpy())

---

In [32]:
tokenizer = AutoTokenizer.from_pretrained('/home/stephen/deepseek/DeepSeek-V3/DeepSeek-V3')

In [33]:
tokens=tokenizer.encode("The American flag is red, white, and")
model.forward(torch.tensor([tokens]), start_pos=0) #Dope! Ran on CPU somehow - took a couple minutes. 

tensor([[0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.bfloat16)

In [34]:
#Do I have non-zero weights now?
model.layers[0].attn.wq_a.weight.shape

torch.Size([1536, 7168])

In [37]:
model.layers[6].attn.wq_a.weight

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.bfloat16,
       requires_grad=True)

In [6]:
start_pos=0 #might need to noodle with this to understad caching
seqlen = torch.tensor([tokens]).size(1)
freqs_cis = model.freqs_cis[start_pos:start_pos+seqlen]

mask = None
if seqlen > 1:
    mask = torch.full((seqlen, seqlen), float("-inf"), device='cpu').triu_(1) #CPUing for now

start_pos, seqlen, freqs_cis.shape, mask.shape

(0, 10, torch.Size([10, 32]), torch.Size([10, 10]))

In [7]:
h = model.embed(torch.tensor([tokens]))

In [8]:
h_out=model.layers[0].attn.forward(h, start_pos, freqs_cis, mask) #Run MLA forward pass from outside

In [9]:
h_out.shape

torch.Size([1, 10, 7168])

### Now Set Through Forward Pass

In [10]:
x=h
mla=model.layers[0].attn

In [11]:
bsz, seqlen, _ = x.size()
end_pos = start_pos + seqlen
if mla.q_lora_rank == 0:
    q = self.wq(x)
else:
    q = mla.wq_b(mla.q_norm(mla.wq_a(x)))
    print('hay')

hay


In [12]:
#Query compression - probably not in script
h1=mla.wq_a(x)
h2=mla.q_norm(mla.wq_a(x))
q=mla.wq_b(mla.q_norm(mla.wq_a(x)))

In [13]:
x.shape, h1.shape, h2.shape, q.shape

(torch.Size([1, 10, 7168]),
 torch.Size([1, 10, 1536]),
 torch.Size([1, 10, 1536]),
 torch.Size([1, 10, 24576]))

In [15]:
q = q.view(bsz, seqlen, mla.n_local_heads, mla.qk_head_dim) #Ok splitting out the queries across all the heads
q_nope, q_pe = torch.split(q, [mla.qk_nope_head_dim, mla.qk_rope_head_dim], dim=-1) #Splitting queries into positional embeddings 
q_pe = apply_rotary_emb(q_pe, freqs_cis)

In [16]:
q.shape, q_nope.shape, q_pe.shape

(torch.Size([1, 10, 128, 192]),
 torch.Size([1, 10, 128, 128]),
 torch.Size([1, 10, 128, 64]))

Ok, this is not that different from GPT-2, for each of our 128 heads, we haev a 10x128 query vector - got it. 

In [18]:
kv = mla.wkv_a(x)

In [20]:
x.shape, kv.shape

(torch.Size([1, 10, 7168]), torch.Size([1, 10, 576]))

- Ok how does this compare to GPT-2 style?